# EDA - User-Items

Nota: es importante realizar los procesos de ETL en la carpeta ./ETL/ y el de Feature Engineering en la carpeta ./Feature Engineering/ para realizar este paso.

En este archivo se desarrollará el EDA completo INDIVIDIAL para los elementos de, archivo resultante de user_reviews.

## Importar Librerías

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys

# Ignorar advertencias
warnings.filterwarnings('ignore')

### importar funciones personalizadas de ../functions/EDA.py

In [2]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

In [28]:
# importar función personalizada de ./VideoGameRecommender/functions/EDA.py
# Esta función permite traer el archivo en el formato que se encuentre (CSV o Parquet)
from functions.EDA import get_file, get_frequency
from functions.ETL import export

## Análisis Exploratorio de Datos

Se realiza el Análisis Exploratorio de Datos en user_items revisando elemento por elemento.

### Revisión general

In [4]:
# utilizamos la función get_file(file_name) importada desde ../functions/EDA.py para traer los datasets
user_items = get_file('user_items')
user_reviews = get_file('user_reviews_sentiment_analysis')
steam_games = get_file('steam_games')

In [5]:
# se verifica si existen user_id en user_reviews que no existan en user_items
missing_user = user_reviews[~user_reviews['user_id'].isin(user_items['user_id'])]
missing_user_quant = get_frequency(missing_user, 'user_id')
print(f'Reviews con usuarios que no figuran en user_items: {len(missing_user_quant)}')
missing_user_quant.head()

Reviews con usuarios que no figuran en user_items: 2845


,user_id,frequency,percentage
0,norsktf,16,0.255346
1,76561198077815717,14,0.223428
2,AkkarinDarkmoor,12,0.191510
3,kiniston69,10,0.159591
4,mshee,10,0.159591


In [6]:
# se filtran los usuarios de user_review que no han hecho

In [7]:
# se hace un merge entre user items y reviews en las columnas item_id y user_id de ambos dataframes
users = pd.merge(user_items, user_reviews, how='outer', on=['item_id', 'user_id'])

# se visualiza y su longitud
print(len(users))
users.head(3)

5169048


,user_id,items_count,item_id,playtime_forever,posted,recommend,review
0,-GM-Dragon,106.0,10,14.0,NaN,NaN,NaN
1,-KillZone-,114.0,10,987.0,NaN,NaN,NaN
2,-Mad-,367.0,10,8.0,NaN,NaN,NaN


In [8]:
# obtener tipo de datos del dataframe
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5169048 entries, 0 to 5169047
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       float64
 2   item_id           int64  
 3   playtime_forever  float64
 4   posted            object 
 5   recommend         float64
 6   review            float64
dtypes: float64(4), int64(1), object(2)
memory usage: 276.1+ MB


In [9]:
# se filtran las filas donde playtime_forever sea 0 y recommend sea nulo
rubbish = users[((users['playtime_forever'] == 0) & (users['recommend'].isna()))]

# ver la cantidad de registros y las primeras filas
print(len(rubbish))
rubbish.head(3)

1867871


,user_id,items_count,item_id,playtime_forever,posted,recommend,review
6,001000111000111000010,71.0,10,0.0,NaN,NaN,NaN
7,00690069006900,102.0,10,0.0,NaN,NaN,NaN
9,010607,28.0,10,0.0,NaN,NaN,NaN


In [10]:
# se hace un merge para obtener las filas que NO están en rubbish
clean_users = users.merge(rubbish, how='left', indicator=True)

# se seleccionan sólo las filas que en _merge es "left_only"
clean_users = clean_users[clean_users['_merge'] == 'left_only']

# se elimina la columna _merge
clean_users.drop(columns='_merge', inplace=True)

print(len(clean_users))
clean_users.head()

3301177


,user_id,items_count,item_id,playtime_forever,posted,recommend,review
0,-GM-Dragon,106.0,10,14.0,NaN,NaN,NaN
1,-KillZone-,114.0,10,987.0,NaN,NaN,NaN
2,-Mad-,367.0,10,8.0,NaN,NaN,NaN
3,-SEVEN-,394.0,10,617.0,NaN,NaN,NaN
4,-SatansLittleHelper-,28.0,10,511.0,NaN,NaN,NaN


In [11]:
#contar cuántos juegos son por usuario
user_game_count = clean_users[clean_users['item_id'].notna()].groupby('user_id')['item_id'].count()

#mostrar distribución
user_game_count.describe()

count    71302.000000
mean        46.298519
std         68.566759
min          1.000000
25%          9.000000
50%         28.000000
75%         61.000000
max       3476.000000
Name: item_id, dtype: float64

### promedio de tiempo de juego y promedio por juego

In [12]:
# Promedio de tiempo de juego por juego
playtime_per_game = clean_users.groupby('item_id')['playtime_forever'].mean()

# Recomendaciones promedio por juego
recommend_per_game = clean_users.groupby('item_id')['recommend'].mean()

# Combinar ambos en un solo DataFrame
game_stats = pd.DataFrame({
    'avg_playtime': playtime_per_game,
    'recommend_rate': recommend_per_game
})

# Ver las primeras filas del nuevo DataFrame
game_stats.head()

,avg_playtime,recommend_rate
item_id,,
10,2879.432759,0.982456
20,366.083746,0.647059
30,599.518957,0.750000
40,231.960961,1.000000
50,384.990566,0.750000


### Unión con el dataframe steam_games

In [13]:
# se eliminan columnas innecesarias (items_count y posted)
clean_users.drop(columns=['items_count', 'posted'], inplace=True)

clean_users.head()

,user_id,item_id,playtime_forever,recommend,review
0,-GM-Dragon,10,14.0,NaN,NaN
1,-KillZone-,10,987.0,NaN,NaN
2,-Mad-,10,8.0,NaN,NaN
3,-SEVEN-,10,617.0,NaN,NaN
4,-SatansLittleHelper-,10,511.0,NaN,NaN


In [14]:
# ver tipo de datos de clean_users
clean_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3301177 entries, 0 to 5209759
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           int64  
 2   playtime_forever  float64
 3   recommend         float64
 4   review            float64
dtypes: float64(3), int64(1), object(1)
memory usage: 151.1+ MB


In [15]:
# Agrupar por item_id y realizar las agregaciones
grouped = clean_users.groupby('item_id').agg({
    'playtime_forever': 'sum',  # Sumar el tiempo total jugado para cada juego
    'recommend': [ 
        ('positive', lambda x: (x == 1).sum()),  # Contar recomendaciones positivas
        ('negative', lambda x: (x == 0).sum())   # Contar recomendaciones negativas
    ],
    'review': [
        ('negative', lambda x: (x == 0).sum()),  # Contar sentimientos negativos
        ('neutral', lambda x: (x == 1).sum()),   # Contar sentimientos neutrales
        ('positive', lambda x: (x == 2).sum())   # Contar sentimientos positivos
    ]
}).reset_index()

# Aplanar las columnas multinivel
grouped.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped.columns]

# Renombrar columnas (si es necesario)
grouped.rename(columns={
    'item_id_': 'id',
    'playtime_forever_sum': 'playtime_total',
    'recommend_positive': 'recommend_pos', 
    'recommend_negative': 'recommend_neg',
    'review_negative': 'review_neg',
    'review_neutral': 'review_neu',
    'review_positive': 'review_pos',
}, inplace=True)

# Imprimir el DataFrame resultante y su longitud
print(len(grouped))
grouped.head()


10415


,id,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos
0,10,17386015.0,56,1,0,4,53
1,20,961702.0,11,6,2,4,11
2,30,758991.0,3,1,0,1,3
3,40,154486.0,1,0,0,0,1
4,50,734562.0,3,1,0,1,3


In [16]:
# se crea un df final y se visualiza
df = pd.merge(steam_games, grouped, on='id', how='left')

print(len(df))
df.head()

212451


,id,app_name,genres,specs,price,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos
0,761140,Lost Summoner Kitty,casual,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
1,761140,Lost Summoner Kitty,action,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
2,761140,Lost Summoner Kitty,indie,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
3,761140,Lost Summoner Kitty,simulation,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
4,761140,Lost Summoner Kitty,strategy,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#contar nulos
df.isnull().sum()

id                     0
app_name               0
genres               483
specs               6691
price               5735
developer              0
release_year       13956
playtime_total    131883
recommend_pos     131883
recommend_neg     131883
review_neg        131883
review_neu        131883
review_pos        131883
dtype: int64

Próximamente sería bueno introducir una función que consiga la media de precios de los juegos de un género y/o spec para luego promediarlo con todos los géneros correspondientes a un título para conseguir un valor no afectado por outliers.

In [18]:
#se reemplazan valores vacíos numéricos y de texto
df['playtime_total'].fillna(0, inplace=True)
df['recommend_pos'].fillna(0, inplace=True)
df['recommend_neg'].fillna(0, inplace=True)
df['review_neg'].fillna(0, inplace=True)
df['review_neu'].fillna(0, inplace=True)
df['review_pos'].fillna(0, inplace=True)
df['release_year'].fillna(0, inplace=True)

# se muestran los nulos
df.isnull().sum()

id                   0
app_name             0
genres             483
specs             6691
price             5735
developer            0
release_year         0
playtime_total       0
recommend_pos        0
recommend_neg        0
review_neg           0
review_neu           0
review_pos           0
dtype: int64

### Tratamiento de los precios

In [19]:
#agrupar las combinaciones de genres y specs y calcular la mediana de los precios
gs_catprice = df.groupby(['developer', 'genres', 'specs']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(gs_catprice))
gs_catprice.head()

99558


,developer,genres,specs,price
0,++,casual,single-player,2.49
1,++,indie,single-player,2.49
2,+7,action,multi-player,4.99
3,+7,action,single-player,4.99
4,+7,arcade,multi-player,4.99


In [20]:
#agrupar las combinaciones de genres y calcular la mediana de los precios (en caso que specs sea nulo)
g_catprice = df.groupby(['developer', 'genres']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(g_catprice))
g_catprice.head()

68061


,developer,genres,price
0,++,casual,2.49
1,++,indie,2.49
2,+7,action,4.99
3,+7,arcade,4.99
4,+7,casual,4.99


In [21]:
#agrupar las combinaciones de specs y calcular la mediana de los precios (en caso que genres sea nulo)
s_catprice = df.groupby(['developer', 'specs']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(s_catprice))
s_catprice.head()

15432


,developer,specs,price
0,++,single-player,2.49
1,+7,multi-player,4.99
2,+7,single-player,4.99
3,+mpact,multi-player,9.99
4,+mpact,online multi-player,9.99


In [22]:
#agrupar las combinaciones de specs y calcular la mediana de los precios (en caso que genres sea nulo)
dev_catprice = df.groupby(['developer']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(dev_catprice))
dev_catprice.head()

10936


,developer,price
0,++,2.49
1,+7,4.99
2,+mpact,9.99
3,07thexpansion,24.99
4,08,6.99


In [23]:
# Merge del dataframe original con el de medianas para obtener las medianas por combinación
merged_df = df.merge(gs_catprice, on=['developer', 'genres', 'specs'], how='left')

# visualizar
merged_df.head()

,id,app_name,genres,specs,price_x,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos,price_y
0,761140,Lost Summoner Kitty,casual,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
1,761140,Lost Summoner Kitty,action,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
2,761140,Lost Summoner Kitty,indie,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
3,761140,Lost Summoner Kitty,simulation,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
4,761140,Lost Summoner Kitty,strategy,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99


In [24]:
# Llenar los valores nulos con las medianas correspondientes
merged_df['price_y'] = merged_df['price_y'].fillna(
    merged_df.merge(s_catprice, on=['developer', 'specs'], how='left')['price_y']
)

merged_df['price_y'] = merged_df['price_y'].fillna(
    merged_df.merge(g_catprice, on=['developer', 'genres'], how='left')['price_y']
)

merged_df['price_y'] = merged_df['price_y'].fillna(
    merged_df.merge(dev_catprice, on='developer', how='left')['price_y']
)

In [25]:
# agrupar por id para calcular la media del precio
final_price = merged_df.groupby('id').agg({
    'price_y': 'mean'  # Media de las combinaciones de géneros y specs
}).reset_index()

final_price.head()

,id,price_y
0,10,12.156667
1,20,8.990000
2,30,9.308182
3,40,8.323333
4,50,13.890000


In [26]:
# crear el df final y ver primeros resultados
df_final = df.merge(final_price, on='id', how='left')

df_final.head()

,id,app_name,genres,specs,price,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos,price_y
0,761140,Lost Summoner Kitty,casual,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
1,761140,Lost Summoner Kitty,action,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
2,761140,Lost Summoner Kitty,indie,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
3,761140,Lost Summoner Kitty,simulation,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99
4,761140,Lost Summoner Kitty,strategy,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,4.99


In [27]:
# reemplazar valores vacíos de price por los elementos de price_y
df_final['price'] = df_final['price'].fillna(df_final['price_y']).round(2)

# eliminar columna price_y y visualizar

df_final.drop(columns='price_y', inplace=True)

df_final.head()

,id,app_name,genres,specs,price,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos
0,761140,Lost Summoner Kitty,casual,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0
1,761140,Lost Summoner Kitty,action,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0
2,761140,Lost Summoner Kitty,indie,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0
3,761140,Lost Summoner Kitty,simulation,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0
4,761140,Lost Summoner Kitty,strategy,single-player,4.99,kotoshiro,2018.0,0.0,0.0,0.0,0.0,0.0,0.0


### Exportar dataframe final

In [31]:
# se utiliza la función export de ../functions/ETL.py para exportar el archivo final
export(df_final, project_root, 'games')

Archivos exportados exitosamente.
